## **5. Dobór strategii agregacji wyników**

#### **Import Bibliotek**

In [2]:
import json
from pathlib import Path
from typing import Dict, Callable, List
import functools

import mlflow
import numpy as np
import pandas as pd
import torch
from databricks.sdk import WorkspaceClient
from datasets import load_from_disk, Dataset, Sequence, Value
from sklearn.metrics import f1_score
from transformers import (
    LongformerForSequenceClassification,
    LongformerTokenizerFast,
    Trainer,
    TrainingArguments,
    default_data_collator,
)

#### **Parametry stałe**

In [3]:
TOKENIZED_DATA_PATH = "data/data_tokenized"
TRAINED_MODEL_PATH = "models/run-21b6e434b35f4028a784f1c0711662a1/final" # Przykładowa ścieżka
MLFLOW_EXPERIMENT_NAME = "ESGAnalyzeModel-Aggregation-Comparison"

CRITERIA_NAMES = [
    'c1_transition_plan', 'c2_risk_management', 'c4_boundaries',
    'c6_historical_data', 'c7_intensity_metrics', 'c8_targets_credibility',
]
NUM_LABELS = len(CRITERIA_NAMES)

#### **Definicje strategii agregacji**

#### Strategia oparta na K-max pooling

In [4]:
def k_max_pooling_aggregation(probs: np.ndarray, k: int) -> np.ndarray:
    """Uśrednia k-najwyższych prawdopodobieństw dla każdej etykiety."""
    # Sortuj prawdopodobieństwa wzdłuż osi fragmentów (axis=0) malejąco
    sorted_probs = np.sort(probs, axis=0)[::-1]
    # Wybierz k-najwyższych i oblicz średnią
    top_k = sorted_probs[:k]
    return np.mean(top_k, axis=0)

#### Strategia oparta na percentylach

In [5]:
def percentile_aggregation(probs: np.ndarray, percentile: int) -> np.ndarray:
    """Oblicza zadany percentyl prawdopodobieństw dla każdej etykiety."""
    return np.percentile(probs, percentile, axis=0)

#### Słownik z parametrami dla strategii

In [6]:
AGGREGATION_STRATEGIES: Dict[str, Callable[[np.ndarray], np.ndarray]] = {
    "percentile_75": functools.partial(percentile_aggregation, percentile=75),
    "percentile_85": functools.partial(percentile_aggregation, percentile=85),
    "percentile_90": functools.partial(percentile_aggregation, percentile=90),
    "percentile_95": functools.partial(percentile_aggregation, percentile=95),
    
    "k_max_pooling_k2": functools.partial(k_max_pooling_aggregation, k=2),
    "k_max_pooling_k3": functools.partial(k_max_pooling_aggregation, k=3),
    "k_max_pooling_k4": functools.partial(k_max_pooling_aggregation, k=4),
    "k_max_pooling_k5": functools.partial(k_max_pooling_aggregation, k=5),
}

#### **Funkcje pomocnicze**

#### Konfiguracja i ustawienie eksperymentu MLflow

In [7]:
def setup_mlflow():
    mlflow.set_tracking_uri("databricks")
    w = WorkspaceClient()
    user_email = w.current_user.me().user_name
    experiment_path = f"/Users/{user_email}/{MLFLOW_EXPERIMENT_NAME}"
    mlflow.set_experiment(experiment_path)
    print(f"Pomyślnie ustawiono eksperyment MLflow: {experiment_path}")

#### Oblicza metryki F1-score na poziomie dokumentów

In [8]:
def calculate_document_metrics(doc_labels: np.ndarray, doc_preds: np.ndarray) -> Dict[str, float]:
    results = {
        'doc_f1_macro': f1_score(doc_labels, doc_preds, average='macro', zero_division=0),
        'num_documents': len(doc_labels)
    }
    f1_per_label = f1_score(doc_labels, doc_preds, average=None, zero_division=0)
    for i, f1 in enumerate(f1_per_label):
        results[f'doc_f1_{CRITERIA_NAMES[i]}'] = f1
    return results

#### **Główna funkcja orkiestrująca proces ewaluacji**

In [9]:
setup_mlflow()

# --- 1. Wczytanie danych i wytrenowanego modelu ---
print("Wczytywanie ztokenizowanych danych...")
tokenized_datasets = load_from_disk(TOKENIZED_DATA_PATH)
eval_dataset = tokenized_datasets['validation']
test_dataset = tokenized_datasets['test']

print("Konwertowanie typu danych etykiet na float32...")
new_features = eval_dataset.features.copy()
new_features['labels'] = Sequence(feature=Value('float32'))
eval_dataset = eval_dataset.cast(new_features)
test_dataset = test_dataset.cast(new_features)

print(f"Wczytywanie wytrenowanego modelu z: {TRAINED_MODEL_PATH}...")
model = LongformerForSequenceClassification.from_pretrained(TRAINED_MODEL_PATH)
tokenizer = LongformerTokenizerFast.from_pretrained(TRAINED_MODEL_PATH)

with open(Path(TRAINED_MODEL_PATH) / "optimal_thresholds.json", "r") as f:
    thresholds_dict = json.load(f)
    optimal_thresholds = np.array([thresholds_dict[name] for name in CRITERIA_NAMES])
print("Wczytano zoptymalizowane progi klasyfikacyjne.")

trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir="./temp_results", per_device_eval_batch_size=4, report_to="none"),
    data_collator=default_data_collator,
    tokenizer=tokenizer,
)

# --- 2. Predykcje na zbiorze walidacyjnym ---
print("Wykonywanie predykcji na zbiorze walidacyjnym...")
preds_output = trainer.predict(eval_dataset)
chunk_probs = 1 / (1 + np.exp(-preds_output.predictions))
print("Predykcje na poziomie fragmentów dla zbioru walidacyjnego zostały wygenerowane.")

df_eval = pd.DataFrame({
    'doc_id': eval_dataset['doc_id'],
    'probs': list(chunk_probs),
    'labels': list(preds_output.label_ids)
})

doc_grouped_eval = df_eval.groupby('doc_id')
doc_labels_map_eval = doc_grouped_eval['labels'].first().to_dict()

# --- 3. Pętla ewaluacyjna na zbiorze walidacyjnym w celu znalezienia najlepszej strategii ---
best_strategy_name = None
best_f1_macro_score = -1.0

for strategy_name, aggregation_func in AGGREGATION_STRATEGIES.items():
    print(f"--- Testowanie strategii na zbiorze walidacyjnym: {strategy_name} ---")
    
    with mlflow.start_run(run_name=f"eval_{strategy_name}") as run:
        mlflow.log_param("aggregation_strategy", strategy_name)
        mlflow.log_param("base_model_path", TRAINED_MODEL_PATH)
        mlflow.log_params({f"threshold_{k}": v for k, v in zip(CRITERIA_NAMES, optimal_thresholds)})

        aggregated_probs = doc_grouped_eval['probs'].apply(
            lambda x: aggregation_func(np.array(x.tolist()))
        )
        
        doc_probs_np = np.array(aggregated_probs.tolist())
        doc_labels_np = np.array(list(doc_labels_map_eval.values()))
        doc_preds_np = (doc_probs_np >= optimal_thresholds).astype(int)
        
        final_metrics = calculate_document_metrics(doc_labels_np, doc_preds_np)
        mlflow.log_metrics(final_metrics)
        
        current_f1 = final_metrics['doc_f1_macro']
        print(f"Wyniki dla '{strategy_name}': F1 Macro (dokument): {current_f1:.4f}")

        # Sprawdzenie i zapisanie najlepszej strategii
        if current_f1 > best_f1_macro_score:
            best_f1_macro_score = current_f1
            best_strategy_name = strategy_name
            print(f"!!! Nowa najlepsza strategia: {best_strategy_name} z F1 Macro: {best_f1_macro_score:.4f} !!!")

print("\n--- Zakończono porównywanie strategii ---")
print(f"Najlepsza znaleziona strategia na zbiorze walidacyjnym: '{best_strategy_name}' (F1 Macro: {best_f1_macro_score:.4f})")

Pomyślnie ustawiono eksperyment MLflow: /Users/adinzlotyint@gmail.com/ESGAnalyzeModel-Aggregation-Comparison
Wczytywanie ztokenizowanych danych...
Konwertowanie typu danych etykiet na float32...


Casting the dataset:   0%|          | 0/1062 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/929 [00:00<?, ? examples/s]

Wczytywanie wytrenowanego modelu z: models/run-21b6e434b35f4028a784f1c0711662a1/final...
Wczytano zoptymalizowane progi klasyfikacyjne.


/tmp/ipykernel_110736/2419543102.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Initializing global attention on CLS token...


Wykonywanie predykcji na zbiorze walidacyjnym...


Predykcje na poziomie fragmentów dla zbioru walidacyjnego zostały wygenerowane.
--- Testowanie strategii na zbiorze walidacyjnym: percentile_75 ---
Wyniki dla 'percentile_75': F1 Macro (dokument): 0.7902
!!! Nowa najlepsza strategia: percentile_75 z F1 Macro: 0.7902 !!!
🏃 View run eval_percentile_75 at: https://dbc-26ad907d-404c.cloud.databricks.com/ml/experiments/4029214961326160/runs/deca2941822b45c08fec55ebdbd26212
🧪 View experiment at: https://dbc-26ad907d-404c.cloud.databricks.com/ml/experiments/4029214961326160
--- Testowanie strategii na zbiorze walidacyjnym: percentile_85 ---
Wyniki dla 'percentile_85': F1 Macro (dokument): 0.7830
🏃 View run eval_percentile_85 at: https://dbc-26ad907d-404c.cloud.databricks.com/ml/experiments/4029214961326160/runs/669ddc26de23428ab7d2251d99babf2c
🧪 View experiment at: https://dbc-26ad907d-404c.cloud.databricks.com/ml/experiments/4029214961326160
--- Testowanie strategii na zbiorze walidacyjnym: percentile_90 ---
Wyniki dla 'percentile_90': F1 Ma

#### **Finalna ewaluacja na zbiorze testowym**

In [10]:
print("Wykonywanie predykcji na zbiorze testowym...")
test_preds_output = trainer.predict(test_dataset)
test_chunk_probs = 1 / (1 + np.exp(-test_preds_output.predictions))

df_test = pd.DataFrame({
    'doc_id': test_dataset['doc_id'],
    'probs': list(test_chunk_probs),
    'labels': list(test_preds_output.label_ids)
})

doc_grouped_test = df_test.groupby('doc_id')
doc_labels_map_test = doc_grouped_test['labels'].first().to_dict()

# Pobranie najlepszej funkcji agregującej
best_aggregation_func = AGGREGATION_STRATEGIES[best_strategy_name]

# Agregacja prawdopodobieństw na zbiorze testowym
aggregated_test_probs = doc_grouped_test['probs'].apply(
    lambda x: best_aggregation_func(np.array(x.tolist()))
)

doc_probs_test_np = np.array(aggregated_test_probs.tolist())
doc_labels_test_np = np.array(list(doc_labels_map_test.values()))

# Zastosowanie progów do predykcji
doc_preds_test_np = (doc_probs_test_np >= optimal_thresholds).astype(int)

# Obliczenie metryk końcowych
final_test_metrics = calculate_document_metrics(doc_labels_test_np, doc_preds_test_np)

print(f"Użyta strategia: {best_strategy_name}")
print(f"  F1 Macro Score (dokument): {final_test_metrics['doc_f1_macro']:.4f}")
print("  Wyniki F1-score dla poszczególnych kryteriów:")
for name in CRITERIA_NAMES:
    metric_key = f'doc_f1_{name}'
    print(f"    - {name}: {final_test_metrics.get(metric_key, 0.0):.4f}")
print("-" * 55)

with mlflow.start_run(run_name="final_test_evaluation") as run:
    mlflow.log_param("best_aggregation_strategy", best_strategy_name)
    mlflow.log_param("base_model_path", TRAINED_MODEL_PATH)
    mlflow.log_params({f"threshold_{k}": v for k, v in zip(CRITERIA_NAMES, optimal_thresholds)})
    
    test_metrics_to_log = {f"test_{k}": v for k, v in final_test_metrics.items()}
    mlflow.log_metrics(test_metrics_to_log)
    print("Wyniki zapisane pomyślnie.")

print("\nZakończono cały proces ewaluacji.")

Wykonywanie predykcji na zbiorze testowym...


Użyta strategia: percentile_75
  F1 Macro Score (dokument): 0.6963
  Wyniki F1-score dla poszczególnych kryteriów:
    - c1_transition_plan: 0.8000
    - c2_risk_management: 0.6207
    - c4_boundaries: 0.6944
    - c6_historical_data: 0.7595
    - c7_intensity_metrics: 0.6364
    - c8_targets_credibility: 0.6667
-------------------------------------------------------
Wyniki zapisane pomyślnie.
🏃 View run final_test_evaluation at: https://dbc-26ad907d-404c.cloud.databricks.com/ml/experiments/4029214961326160/runs/3414674617434849972182960f481d30
🧪 View experiment at: https://dbc-26ad907d-404c.cloud.databricks.com/ml/experiments/4029214961326160

Zakończono cały proces ewaluacji.
